In [ ]:
import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

In [ ]:
"""Load support tickets data from CSV into PostgreSQL database."""
import csv
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
from agent_tools import *


load_dotenv(os.path.join(parent_dir, ".env"))

In [ ]:
# search_similar_issues("WiFi keeps dropping")
search_similar_issues("Slow internet connection")

### Create the agent with tool access
We created the agent in the Azure AI Foundry project and added the Postgres tools needed to query the database.

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import FilePurpose, CodeInterpreterTool, ListSortOrder, MessageRole
from azure.ai.agents.models import FunctionTool, ToolSet
from datetime import datetime

In [ ]:
# Create agent client
credential = DefaultAzureCredential(exclude_environment_credential=True,
         exclude_managed_identity_credential=True)
project_client = AIProjectClient(
    credential=credential,
    endpoint=os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT"),
)
project_client 

In [ ]:
agents_client = project_client.agents
agents_client.enable_auto_function_calls(user_functions)


In [ ]:
instructions = """
You are a helpful legal assistant who can retrieve information about service tickets from postgres database.
Select the following tools to answer the user's question:
- search_similar_issues(issue_description: str, limit: int = 5) -> List[Dict]
    : Searches for similar issues in the support tickets database based on the provided issue description.

Don't return the tickets not matching with user query.
"""

In [ ]:
# Initialize the agent toolset with user functions
functions = FunctionTool(user_functions)
# toolset = ToolSet()
# toolset.add(functions)

In [ ]:
# Define an agent that uses the code ai_search tool
agent = project_client.agents.create_agent(
    name="TicketAnalyzer-Agent",
    model=os.environ["AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME"],  # Model deployment name 4o
    description="An agent that can analyze and retrieve support tickets from Postgres database.",
    instructions=instructions,
    tools=functions.definitions,
)

print(f"Created agent, ID: {agent.id}")

In [ ]:
# Create thread for communication
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")

In [ ]:
# Create message to thread
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you find support tickets related to login issues? Use the available tools.",
)
print(f"Created message, ID: {message.id}")

In [ ]:
# Create and process agent run in thread with tools
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

In [ ]:
# Fetch and log all messages
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")